In [1]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [41]:
import pandas as pd

df_temp=pd.read_csv('gdrive/MyDrive/Data/GlobalLandTemperaturesByCountry.csv') #Temperature df
df_co2=pd.read_csv('gdrive/MyDrive/Data/CO2emissionbycountries.csv',encoding= 'unicode_escape') #CO2 df

In [42]:
df_temp.head(5)

,dt,AverageTemperature,AverageTemperatureUncertainty,Country
0,1743-11-01,4.384,2.294,Åland
1,1743-12-01,NaN,NaN,Åland
2,1744-01-01,NaN,NaN,Åland
3,1744-02-01,NaN,NaN,Åland
4,1744-03-01,NaN,NaN,Åland


In [43]:
pd.DataFrame(df_temp)


,dt,AverageTemperature,AverageTemperatureUncertainty,Country
0,1743-11-01,4.384,2.294,Åland
1,1743-12-01,NaN,NaN,Åland
2,1744-01-01,NaN,NaN,Åland
3,1744-02-01,NaN,NaN,Åland
4,1744-03-01,NaN,NaN,Åland
...,...,...,...,...
577457,2013-05-01,19.059,1.022,Zimbabwe
577458,2013-06-01,17.613,0.473,Zimbabwe
577459,2013-07-01,17.000,0.453,Zimbabwe
577460,2013-08-01,19.759,0.717,Zimbabwe


In [5]:
df_co2.head(5)

,Country,Code,Calling Code,Year,CO2 emission (Tons),Population(2022),Area,% of World,Density(km2)
0,Afghanistan,AF,93,1750,0.0,41128771.0,652230.0,0.40%,63/km²
1,Afghanistan,AF,93,1751,0.0,41128771.0,652230.0,0.40%,63/km²
2,Afghanistan,AF,93,1752,0.0,41128771.0,652230.0,0.40%,63/km²
3,Afghanistan,AF,93,1753,0.0,41128771.0,652230.0,0.40%,63/km²
4,Afghanistan,AF,93,1754,0.0,41128771.0,652230.0,0.40%,63/km²


In [6]:
#Dropping data with CO2 emission equal 0
df= df_co2[df_co2['CO2 emission (Tons)'] != 0]

In [7]:
print(df)

           Country Code Calling Code  Year  CO2 emission (Tons)  \
199    Afghanistan   AF           93  1949              14656.0   
200    Afghanistan   AF           93  1950              98928.0   
201    Afghanistan   AF           93  1951             190528.0   
202    Afghanistan   AF           93  1952             282128.0   
203    Afghanistan   AF           93  1953             388384.0   
...            ...  ...          ...   ...                  ...   
59615     Zimbabwe   ZW          263  2016          736467042.0   
59616     Zimbabwe   ZW          263  2017          746048675.0   
59617     Zimbabwe   ZW          263  2018          757903042.0   
59618     Zimbabwe   ZW          263  2019          768852126.0   
59619     Zimbabwe   ZW          263  2020          779383468.0   

       Population(2022)      Area % of World Density(km2)  
199          41128771.0  652230.0      0.40%       63/km²  
200          41128771.0  652230.0      0.40%       63/km²  
201          41

In [8]:
#Preparing data by dropping duplicates to visualize in which years surveys start.
df_single=df
df_single=df_single.drop_duplicates(subset=['Country'])
print(df_single)

                 Country Code Calling Code  Year  CO2 emission (Tons)  \
199          Afghanistan   AF           93  1949              14656.0   
454              Albania   AL          355  1933               7328.0   
708              Algeria   DZ          213  1916               3664.0   
1053             Andorra   AD          376  1990             406704.0   
1284              Angola   AO          244  1950             186864.0   
...                  ...  ...          ...   ...                  ...   
58407            Vietnam   VN           84  1892             212512.0   
58787  Wallis and Futuna   WF          681  2001              14656.0   
59007              Yemen   YE          967  1950              58624.0   
59278             Zambia   ZM          260  1950            2330187.0   
59502           Zimbabwe   ZW          263  1903             113584.0   

       Population(2022)       Area % of World Density(km2)  
199          41128771.0   652230.0      0.40%       63/km²  
4

In [9]:
#Filtering country with data surveys starting from 1959
df_single = df_single[df_single['Year'] > 1959]
print(df_single)

                        Country Code Calling Code  Year  CO2 emission (Tons)  \
1053                    Andorra   AD          376  1990             406704.0   
1595                   Anguilla   AI        1-264  1990              51296.0   
1863                 Antarctica   AQ          672  1987               3664.0   
6453                     Bhutan   BT          975  1970               3664.0   
7268                   Botswana   BW          267  1972              21984.0   
11060          Christmas Island   CX           61  1970              69616.0   
12143              Cook Islands   CK          682  1969              10992.0   
27582                  Kiribati   KI          686  1961               7328.0   
27900                    Kosovo   XK          383  2008            7386624.0   
29508                   Lesotho   LS          266  1990            1472928.0   
30321             Liechtenstein   LI          423  1990             198968.0   
32199                  Maldives   MV    

In [36]:
#Left outer join exclusion on country with data surveys starting from 1959
df_final=pd.merge(df,df_single[['Country']],on=['Country'],how="outer",indicator=True)
df_final=df_final[df_final['_merge']=='left_only']
# resetting the DataFrame index
df_final = df_final.reset_index()

In [53]:
df_new=df_final[df_final['Year'] >= 1959]

df_new = df_new.reset_index()

In [54]:
df_new

,level_0,index,Country,Code,Calling Code,Year,CO2 emission (Tons),Population(2022),Area,% of World,Density(km2),_merge
0,10,10,Afghanistan,AF,93,1959,1839179.0,41128771.0,652230.0,0.40%,63/km²,left_only
1,11,11,Afghanistan,AF,93,1960,2253064.0,41128771.0,652230.0,0.40%,63/km²,left_only
2,12,12,Afghanistan,AF,93,1961,2743862.0,41128771.0,652230.0,0.40%,63/km²,left_only
3,13,13,Afghanistan,AF,93,1962,3432456.0,41128771.0,652230.0,0.40%,63/km²,left_only
4,14,14,Afghanistan,AF,93,1963,4139192.0,41128771.0,652230.0,0.40%,63/km²,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...
11837,20533,21702,Zimbabwe,ZW,263,2016,736467042.0,16320537.0,390757.0,0.30%,42/km²,left_only
11838,20534,21703,Zimbabwe,ZW,263,2017,746048675.0,16320537.0,390757.0,0.30%,42/km²,left_only
11839,20535,21704,Zimbabwe,ZW,263,2018,757903042.0,16320537.0,390757.0,0.30%,42/km²,left_only
11840,20536,21705,Zimbabwe,ZW,263,2019,768852126.0,16320537.0,390757.0,0.30%,42/km²,left_only


In [21]:
df_final.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20538 entries, 0 to 21706
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   Country              20538 non-null  object  
 1   Code                 19874 non-null  object  
 2   Calling Code         19498 non-null  object  
 3   Year                 20538 non-null  int64   
 4   CO2 emission (Tons)  20538 non-null  float64 
 5   Population(2022)     19378 non-null  float64 
 6   Area                 19668 non-null  float64 
 7   % of World           19668 non-null  object  
 8   Density(km2)         19378 non-null  object  
 9   _merge               20538 non-null  category
dtypes: category(1), float64(3), int64(1), object(5)
memory usage: 2.1+ MB


In [61]:
df_new=df_new.drop(columns=['index','level_0','Code','Calling Code','_merge','Area','Population(2022)','% of World'])

In [62]:
df_new.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11842 entries, 0 to 11841
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Country              11842 non-null  object 
 1   Year                 11842 non-null  int64  
 2   CO2 emission (Tons)  11842 non-null  float64
 3   Density(km2)         10974 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 370.2+ KB


In [92]:

check_nan =df_new['Density(km2)'].isnull()
print(check_nan)
df_new.loc[df_new['Density(km2)'].isnull()]

# printing the result

0        False
1        False
2        False
3        False
4        False
         ...  
11837    False
11838    False
11839    False
11840    False
11841    False
Name: Density(km2), Length: 11842, dtype: bool


,Country,Year,CO2 emission (Tons),Density(km2)
434,Aruba,1959,10044560.0,NaN
435,Aruba,1960,10663121.0,NaN
436,Aruba,1961,11308674.0,NaN
437,Aruba,1962,12017616.0,NaN
438,Aruba,1963,12696704.0,NaN
...,...,...,...,...
9605,Sint Maarten (Dutch part),2016,64902615.0,NaN
9606,Sint Maarten (Dutch part),2017,65653735.0,NaN
9607,Sint Maarten (Dutch part),2018,66375543.0,NaN
9608,Sint Maarten (Dutch part),2019,67080068.0,NaN
